In [1]:
from matplotlib import pyplot as plt
from mxnet import autograd, nd, init, gluon
from mxnet.gluon import data as gdata
from mxnet.gluon import loss as gloss
from mxnet.gluon import nn, rnn
from time import time
import mxnet as mx
import d2lzh as d2l
import random
import zipfile
import math
import time
import sys 

# 06. 循环神经⽹络
与多层感知机和能有效处理空间信息的卷积神经⽹络不同，循环神经⽹络是为更好地处理时序信息而设计的。它引⼊状态变量来存储过去的信息，并⽤其与当前的输⼊共同决定当前的输出。

循环神经⽹络常⽤于处理序列数据，如⼀段⽂字或声⾳、购物或观影的顺序，甚⾄是图像中的⼀⾏或⼀列像素。因此，循环神经⽹络有着极为⼴泛的实际应⽤，如语⾔模型、⽂本分类、机器翻译、语⾳识别、图像分析、⼿写识别和推荐系统。

因为本章中的应⽤是基于语⾔模型的，所以我们将先介绍语⾔模型的基本概念，并由此激发循环神经⽹络的设计灵感。接着，我们将描述循环神经⽹络中的梯度计算⽅法，从而探究循环神经⽹络训练可能存在的问题。对于其中的部分问题，我们可以使⽤本章稍后介绍的含⻔控的循环神经⽹络来解决。最后，我们将拓展循环神经⽹络的架构。

## 6.1 语言模型
`语言模型`(language model)是自然语言处理的重要技术。自然语言处理中最常见的数据是文本数据。我们可以把一段自然语言文本看作一段离散的时间序列。假设一段长度为$T$的文本中的词依次为$w_1, w_2, \ldots, w_T$，那么在离散的时间序列中，$w_t$($1 \leq t \leq T$)可看作在`时间步`(time step)$t$的输出或标签。

给定一个长度为$T$的词的序列$w_1, w_2, \ldots, w_T$，语言模型将计算该序列的概率：
+ $P(w_1, w_2, \ldots, w_T)$

语言模型可用于提升语音识别和机器翻译的性能。例如，如果对英文“you go first”逐词翻译成中文的话，可能得到“你走先”“你先走”等排列方式的文本序列。如果语言模型判断出“你先走”的概率大于其他排列方式的文本序列的概率，我们就可以把“you go first”翻译成“你先走”。

### 6.1.1 语言模型的计算
该如何计算语言模型呢？假设序列$w_1, w_2, \ldots, w_T$中的每个词是依次生成的，我们有
+ $P(w_1, w_2, \ldots, w_T) = \prod_{t=1}^T P(w_t \mid w_1, \ldots, w_{t-1})$

例如，一段含有4个词的文本序列的概率
+ $P(w_1, w_2, w_3, w_4) = P(w_1) P(w_2 \mid w_1) P(w_3 \mid w_1, w_2) P(w_4 \mid w_1, w_2, w_3)$

为了计算语言模型，我们需要计算词的概率，以及一个词在给定前几个词的情况下的条件概率，即`语言模型参数`。设训练数据集为一个大型文本语料库，如维基百科的所有条目。词的概率可以通过该词在训练数据集中的相对词频来计算。例如，$P(w_1)$可以计算为$w_1$在训练数据集中的词频与训练数据集的总词数之比。因此，根据条件概率定义，一个词在给定前几个词的情况下的条件概率也可以通过训练数据集中的相对词频计算。例如，$P(w_2 \mid w_1)$可以计算为$w_1, w_2$两词相邻的频率与$w_1$词频的比值，因为该比值即$P(w_1, w_2)$与$P(w_1)$之比；而$P(w_3 \mid w_1, w_2)$同理可以计算为$w_1$、$w_2$和$w_3$三词相邻的频率与$w_1$和$w_2$两词相邻的频率的比值。以此类推。

### 6.1.2 n元语法
当序列长度增加时，计算和存储多个词共同出现的概率的复杂度会呈指数级增加。$n$元语法通过`马尔可夫假设`(虽然并不一定成立)简化了语言模型的计算。这里的马尔可夫假设是指一个词的出现只与前面$n$个词相关，即$n$阶`马尔可夫链`(Markov chain of order $n$)。如果$n=1$，那么有$P(w_3 \mid w_1, w_2) = P(w_3 \mid w_2)$。如果基于$n-1$阶马尔可夫链，我们可以将语言模型改写为
+ $P(w_1, w_2, \ldots, w_T) \approx \prod_{t=1}^T P(w_t \mid w_{t-(n-1)}, \ldots, w_{t-1})$

以上也叫$n$元语法($n$-grams)。它是基于$n - 1$阶马尔可夫链的概率语言模型。当$n$分别为1、2和3时，我们将其分别称作一元语法(unigram)、二元语法(bigram)和三元语法(trigram)。例如，长度为4的序列$w_1, w_2, w_3, w_4$在一元语法、二元语法和三元语法中的概率分别为
$$ \begin{aligned} P(w_1, w_2, w_3, w_4) &= P(w_1) P(w_2) P(w_3) P(w_4) ,\\ P(w_1, w_2, w_3, w_4) &= P(w_1) P(w_2 \mid w_1) P(w_3 \mid w_2) P(w_4 \mid w_3) ,\\ P(w_1, w_2, w_3, w_4) &= P(w_1) P(w_2 \mid w_1) P(w_3 \mid w_1, w_2) P(w_4 \mid w_2, w_3) . \end{aligned} $$

当$n$较小时，$n$元语法往往并不准确。例如，在一元语法中，由三个词组成的句子“你走先”和“你先走”的概率是一样的。然而，当$n$较大时，$n$元语法需要计算并存储大量的词频和多词相邻频率。

那么，有没有方法在语言模型中更好地平衡以上这两点呢？我们将在本章探究这样的方法。

## 6.2 循环神经网络
上一节介绍的$n$元语法中，时间步$t$的词$w_t$基于前面所有词的条件概率只考虑了最近时间步的$n-1$个词。如果要考虑比$t-(n-1)$更早时间步的词对$w_t$的可能影响，我们需要增大$n$。但这样模型参数的数量将随之呈指数级增长。

本节将介绍循环神经网络。它并非刚性地记忆所有固定长度的序列，而是通过隐藏状态来存储之前时间步的信息。我们通过多层感知机描述如何添加隐藏状态来将它变成循环神经网络。

### 6.2.1 不含隐藏状态的神经网络
让我们考虑一个含单隐藏层的多层感知机。给定样本数为$n$、输入个数(特征数或特征向量维度)为$d$的小批量数据样本$\boldsymbol{X} \in \mathbb{R}^{n \times d}$。设隐藏层的激活函数为$\phi$，那么隐藏层的输出$\boldsymbol{H} \in \mathbb{R}^{n \times h}$计算为
+ $\boldsymbol{H} = \phi(\boldsymbol{X} \boldsymbol{W}_{xh} + \boldsymbol{b}_h)$

其中隐藏层权重参数$\boldsymbol{W}_{xh} \in \mathbb{R}^{d \times h}$，隐藏层偏差参数 $\boldsymbol{b}_h \in \mathbb{R}^{1 \times h}$，$h$为隐藏单元个数。把隐藏变量$\boldsymbol{H}$作为输出层的输入，且设输出个数为$q$(如分类问题中的类别数)，输出层的输出为
+ $\boldsymbol{O} = \boldsymbol{H} \boldsymbol{W}_{hq} + \boldsymbol{b}_q$

其中输出变量$\boldsymbol{O} \in \mathbb{R}^{n \times q}$, 输出层权重参数$\boldsymbol{W}_{hq} \in \mathbb{R}^{h \times q}$, 输出层偏差参数$\boldsymbol{b}_q \in \mathbb{R}^{1 \times q}$。如果是分类问题，我们可以使用$\text{softmax}(\boldsymbol{O})$来计算输出类别的概率分布。

### 6.2.2 含隐藏状态的循环神经网络
现在我们考虑输入数据存在时间相关性的情况。假设$\boldsymbol{X}_t \in \mathbb{R}^{n \times d}$是序列中时间步$t$的小批量输入，$\boldsymbol{H}_t \in \mathbb{R}^{n \times h}$是该时间步的隐藏变量。与多层感知机不同的是，这里我们保存上一时间步的隐藏变量$\boldsymbol{H}_{t-1}$，并引入一个新的权重参数$\boldsymbol{W}_{hh} \in \mathbb{R}^{h \times h}$，该参数用来描述在当前时间步如何使用上一时间步的隐藏变量。具体来说，时间步$t$的隐藏变量的计算由当前时间步的输入和上一时间步的隐藏变量共同决定：
+ $\boldsymbol{H}_t = \phi(\boldsymbol{X}_t \boldsymbol{W}_{xh} + \boldsymbol{H}_{t-1} \boldsymbol{W}_{hh} + \boldsymbol{b}_h)$

与多层感知机相比，我们在这里添加了$\boldsymbol{H}_{t-1} \boldsymbol{W}_{hh}$一项。由上式中相邻时间步的隐藏变量$\boldsymbol{H}_t$和$\boldsymbol{H}_{t-1}$之间的关系可知，这里的隐藏变量能够捕捉截至当前时间步的序列的历史信息，就像是神经网络当前时间步的状态或记忆一样。因此，该隐藏变量也称为`隐藏状态`。由于隐藏状态在当前时间步的定义使用了上一时间步的隐藏状态，上式的计算是循环的。使用循环计算的网络即`循环神经网络`(recurrent neural network)。

循环神经网络有很多种不同的构造方法。若无特别说明，本章中的循环神经网络均基于上式中隐藏状态的循环计算。

在时间步$t$，输出层的输出和多层感知机中的计算类似：
+ $\boldsymbol{O}_t = \boldsymbol{H}_t \boldsymbol{W}_{hq} + \boldsymbol{b}_q$

循环神经网络的参数包括隐藏层的权重$\boldsymbol{W}_{xh} \in \mathbb{R}^{d \times h}$、$\boldsymbol{W}_{hh} \in \mathbb{R}^{h \times h}$和偏差 $\boldsymbol{b}_h \in \mathbb{R}^{1 \times h}$，以及输出层的权重$\boldsymbol{W}_{hq} \in \mathbb{R}^{h \times q}$和偏差$\boldsymbol{b}_q \in \mathbb{R}^{1 \times q}$。值得一提的是，即便在不同时间步，循环神经网络也始终使用这些模型参数。因此，循环神经网络模型参数的数量不随时间步的增加而增长。

`图6.1`展示了循环神经网络在3个相邻时间步的计算逻辑。在时间步$t$，隐藏状态的计算可以看成是将输入$\boldsymbol{X}_t$和前一时间步隐藏状态$\boldsymbol{H}_{t-1}$连结后输入一个激活函数为$\phi$的全连接层。该全连接层的输出就是当前时间步的隐藏状态$\boldsymbol{H}_t$，且模型参数为$\boldsymbol{W}_{xh}$与$\boldsymbol{W}_{hh}$的连结，偏差为$\boldsymbol{b}_h$。当前时间步$t$的隐藏状态$\boldsymbol{H}_t$将参与下一个时间步$t+1$的隐藏状态$\boldsymbol{H}_{t+1}$的计算，并输入到当前时间步的全连接输出层。

<img src="images/06_01.png" style="width:600px;"/>

其中，隐藏状态中$\boldsymbol{X}_t \boldsymbol{W}_{xh} + \boldsymbol{H}_{t-1} \boldsymbol{W}_{hh}$的计算等价于$\boldsymbol{X}_t$与$\boldsymbol{H}_{t-1}$连结后的矩阵乘以$\boldsymbol{W}_{xh}$与$\boldsymbol{W}_{hh}$连结后的矩阵。接下来，我们用一个具体的例子来验证这一点。首先，我们构造矩阵$X$、$W_{xh}$、$H$和$W_{hh}$，它们的形状分别为(3, 1)、(1, 4)、(3, 4)和(4, 4)。将$X$与$W_{xh}$、$H$与$W_{hh}$分别相乘，再把两个乘法运算的结果相加，得到形状为(3, 4)的矩阵。

In [2]:
X, W_xh = nd.random.normal(shape=(3, 1)), nd.random.normal(shape=(1, 4))
H, W_hh = nd.random.normal(shape=(3, 4)), nd.random.normal(shape=(4, 4))
nd.dot(X, W_xh) + nd.dot(H, W_hh)


[[ 5.0373516   2.6754622  -1.6607479  -0.40628886]
 [ 0.948454    0.46941757 -1.1866101  -1.180677  ]
 [-1.151402    0.83730245 -2.1974368  -5.248016  ]]
<NDArray 3x4 @cpu(0)>

将矩阵$X$和$H$按列(维度1)连结，连结后的矩阵形状为(3, 5)。可见，连结后矩阵在维度1的长度为矩阵$X$和$H$在维度1的长度之和($1+4$)。然后，将矩阵$W_{xh}$和$W_{hh}$按行(维度0)连结，连结后的矩阵形状为(5, 4)。最后将两个连结后的矩阵相乘，得到与上面代码输出相同的形状为(3, 4)的矩阵。

In [3]:
nd.dot(nd.concat(X, H, dim=1), nd.concat(W_xh, W_hh, dim=0))


[[ 5.0373516   2.6754622  -1.6607479  -0.40628862]
 [ 0.94845396  0.46941754 -1.1866102  -1.1806769 ]
 [-1.151402    0.8373025  -2.1974368  -5.248016  ]]
<NDArray 3x4 @cpu(0)>

### 6.2.3 应用：基于字符级循环神经网络的语言模型
最后我们介绍如何应用循环神经网络来构建一个语言模型。设小批量中样本数为1，文本序列为`想`、`要`、`有`、`直`、`升`、`机`。`图6.2`演示了如何使用循环神经网络基于当前和过去的字符来预测下一个字符。在训练时，我们对每个时间步的输出层输出使用softmax运算，然后使用交叉熵损失函数来计算它与标签的误差。在`图6.2`中，由于隐藏层中隐藏状态的循环计算，时间步3的输出$\boldsymbol{O}_3$取决于文本序列`想`、`要`、`有`。 由于训练数据中该序列的下一个词为`直`，时间步3的损失将取决于该时间步基于序列`想`、`要`、`有`生成下一个词的概率分布与该时间步的标签`直`。

<img src="images/06_02.png" style="width:600px;"/>

因为每个输入词是一个字符，因此这个模型被称为`字符级循环神经网络`(character-level recurrent neural network)。因为不同字符的个数远小于不同词的个数，所以字符级循环神经网络的计算通常更加简单。在接下来的几节里，我们将介绍它的具体实现。


## 6.3 语⾔模型数据集(周杰伦专辑歌词)
本节将介绍如何预处理一个语言模型数据集，并将其转换成字符级循环神经网络所需要的输入格式。为此，我们收集了周杰伦十张专辑中的歌词，并在后面几节里应用循环神经网络来训练一个语言模型。当模型训练好后，我们就可以用这个模型来创作歌词。

### 6.3.1 读取数据集
首先读取这个数据集，看看前40个字符是什么样的。

In [4]:
with zipfile.ZipFile('./data/jaychou_lyrics.txt.zip') as zin:
    with zin.open('jaychou_lyrics.txt') as f:
        corpus_chars = f.read().decode('utf-8') 
corpus_chars[:40]

'想要有直升机\n想要和你飞到宇宙去\n想要和你融化在一起\n融化在宇宙里\n我每天每天每'

这个数据集有6万多个字符。为了打印方便，我们把换行符替换成空格，然后仅使用前1万个字符来训练模型。

In [5]:
corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ') 
corpus_chars = corpus_chars[0:10000]

### 6.3.2 建立字符索引
我们将每个字符映射成一个从0开始的连续整数，又称索引，来方便之后的数据处理。为了得到索引，我们将数据集里所有不同字符取出来，然后将其逐一映射到索引来构造词典。

In [6]:
idx_to_char = list(set(corpus_chars))
char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)]) 
vocab_size = len(char_to_idx) 
vocab_size

1027

之后，将训练数据集中每个字符转化为索引，并打印前20个字符及其对应的索引。

In [7]:
corpus_indices = [char_to_idx[char] for char in corpus_chars] 
sample = corpus_indices[:20] 
print('chars:', ''.join([idx_to_char[idx] for idx in sample])) 
print('indices:', sample)

chars: 想要有直升机 想要和你飞到宇宙去 想要和
indices: [704, 197, 914, 294, 495, 99, 550, 704, 197, 270, 69, 191, 304, 744, 517, 580, 550, 704, 197, 270]


我们将以上代码封装在`d2lzh`包⾥的`load_data_jay_lyrics`函数中。调⽤该函数后会依次得到`corpus_indices`、`char_to_idx`、`idx_to_char`和`vocab_size`。

### 6.3.3 时序数据的采样
时序数据的一个样本通常包含连续的字符。假设时间步数为5，样本序列为5个字符，即`想`、`要`、`有`、`直`、`升`。该样本的标签序列为这些字符分别在训练集中的下一个字符，即`要`、`有`、`直`、`升`、`机`。我们有两种方式对时序数据进行采样，分别是随机采样和相邻采样。

##### 随机采样
下面的代码每次从数据里随机采样一个小批量。其中批量大小`batch_size`指每个小批量的样本数，`num_steps`为每个样本所包含的时间步数。在随机采样中，每个样本是原始序列上任意截取的一段序列。相邻的两个随机小批量在原始序列上的位置不一定相毗邻。因此，我们无法用一个小批量最终时间步的隐藏状态来初始化下一个小批量的隐藏状态。在训练模型时，每次随机采样前都需要重新初始化隐藏状态。

In [8]:
# 本函数已保存在d2lzh包中⽅便以后使⽤
def data_iter_random(corpus_indices, batch_size, num_steps, ctx=None):
    # 减1是因为输出的索引是相应输⼊的索引加1 
    num_examples = (len(corpus_indices) - 1) // num_steps 
    epoch_size = num_examples // batch_size 
    example_indices = list(range(num_examples)) 
    random.shuffle(example_indices)

    # 返回从pos开始的⻓为num_steps的序列
    def _data(pos):
        return corpus_indices[pos: pos + num_steps]

    for i in range(epoch_size):
        # 每次读取batch_size个随机样本 
        i = i * batch_size 
        batch_indices = example_indices[i: i + batch_size] 
        X = [_data(j * num_steps) for j in batch_indices] 
        Y = [_data(j * num_steps + 1) for j in batch_indices] 
        yield nd.array(X, ctx), nd.array(Y, ctx)

让我们输⼊⼀个从0到29的连续整数的⼈⼯序列。设批量⼤小和时间步数分别为2和6。打印随机采样每次读取的小批量样本的输⼊$X$和标签$Y$。可⻅，相邻的两个随机小批量在原始序列上的位置不⼀定相毗邻。

In [9]:
my_seq = list(range(30))
for X, Y in data_iter_random(my_seq, batch_size=2, num_steps=6):
    print('X: ', X, '\nY:', Y, '\n')

X:  
[[ 0.  1.  2.  3.  4.  5.]
 [18. 19. 20. 21. 22. 23.]]
<NDArray 2x6 @cpu(0)> 
Y: 
[[ 1.  2.  3.  4.  5.  6.]
 [19. 20. 21. 22. 23. 24.]]
<NDArray 2x6 @cpu(0)> 

X:  
[[ 6.  7.  8.  9. 10. 11.]
 [12. 13. 14. 15. 16. 17.]]
<NDArray 2x6 @cpu(0)> 
Y: 
[[ 7.  8.  9. 10. 11. 12.]
 [13. 14. 15. 16. 17. 18.]]
<NDArray 2x6 @cpu(0)> 



##### 相邻采样
除对原始序列做随机采样之外，我们还可以令相邻的两个随机小批量在原始序列上的位置相毗邻。我们⽤⼀个小批量最终时间步的隐藏状态来初始化下⼀个小批量的隐藏状态，从而使下⼀个小批量的输出也取决于当前小批量的输⼊，并如此循环下去。

相邻采样对循环神经⽹络造成了两⽅⾯影响：一方面，在训练模型时，我们只需在每一个迭代周期开始时初始化隐藏状态；另一方面，当多个相邻小批量通过传递隐藏状态串联起来时，模型参数的梯度计算将依赖所有串联起来的小批量序列。同一迭代周期中，随着迭代次数的增加，梯度的计算开销会越来越大。为了使模型参数的梯度计算只依赖一次迭代读取的小批量序列，我们可以在每次读取小批量前将隐藏状态从计算图中分离出来。

我们将在`循环神经网络的从零开始实现`一节的实现中了解这种处理方式。

In [10]:
# 本函数已保存在d2lzh包中⽅便以后使⽤
def data_iter_consecutive(corpus_indices, batch_size, num_steps, ctx=None):
    corpus_indices = nd.array(corpus_indices, ctx=ctx) 
    data_len = len(corpus_indices) 
    batch_len = data_len // batch_size 
    indices = corpus_indices[0: batch_size*batch_len].reshape(( batch_size, batch_len))
    epoch_size = (batch_len - 1) // num_steps 
    for i in range(epoch_size):
        i = i * num_steps
        X = indices[:, i: i + num_steps]
        Y = indices[:, i + 1: i + num_steps + 1] 
        yield X, Y

同样的设置下，相邻的两个随机小批量在原始序列上的位置相毗邻。

In [11]:
for X, Y in data_iter_consecutive(my_seq, batch_size=2, num_steps=6): 
    print('X: ', X, '\nY:', Y, '\n')

X:  
[[ 0.  1.  2.  3.  4.  5.]
 [15. 16. 17. 18. 19. 20.]]
<NDArray 2x6 @cpu(0)> 
Y: 
[[ 1.  2.  3.  4.  5.  6.]
 [16. 17. 18. 19. 20. 21.]]
<NDArray 2x6 @cpu(0)> 

X:  
[[ 6.  7.  8.  9. 10. 11.]
 [21. 22. 23. 24. 25. 26.]]
<NDArray 2x6 @cpu(0)> 
Y: 
[[ 7.  8.  9. 10. 11. 12.]
 [22. 23. 24. 25. 26. 27.]]
<NDArray 2x6 @cpu(0)> 



## 6.4 循环神经⽹络的从零开始实现
我们将从零开始实现⼀个基于字符级循环神经⽹络的语⾔模型，并在周杰伦专辑歌词数据集上训练⼀个模型来进⾏歌词创作。⾸先，我们读取周杰伦专辑歌词数据集：

In [12]:
def load_data_jay_lyrics():
    """Load the Jay Chou lyric data set (available in the Chinese book)."""
    with zipfile.ZipFile('./data/jaychou_lyrics.txt.zip') as zin:
        with zin.open('jaychou_lyrics.txt') as f:
            corpus_chars = f.read().decode('utf-8')
    corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
    corpus_chars = corpus_chars[0:10000]
    idx_to_char = list(set(corpus_chars))
    char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
    vocab_size = len(char_to_idx)
    corpus_indices = [char_to_idx[char] for char in corpus_chars]
    return corpus_indices, char_to_idx, idx_to_char, vocab_size

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = load_data_jay_lyrics()

### 6.4.1 one-hot向量
为了将词表示成向量输入到神经网络，一个简单的办法是使用one-hot向量。

下面分别展示了索引为0和2的one-hot向量，向量长度等于词典大小。

In [13]:
nd.one_hot(nd.array([0, 2]), vocab_size)


[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]
<NDArray 2x1027 @cpu(0)>

我们每次采样的小批量的形状是(批量大小, 时间步数)。下面的函数将这样的小批量变换成数个可以输入进网络的形状为(批量大小, 词典大小)的矩阵，矩阵个数等于时间步数。也就是说，时间步$t$的输入为$\boldsymbol{X}_t \in \mathbb{R}^{n \times d}$，其中$n$为批量大小，$d$为输入个数，即one-hot向量长度(词典大小)。

In [14]:
# 本函数已保存在d2lzh包中⽅便以后使⽤ 
def to_onehot(X, size): 
    return [nd.one_hot(x, size) for x in X.T]

X = nd.arange(10).reshape((2, 5)) 
inputs = to_onehot(X, vocab_size) 
# inputs = to_onehot(X, 10) 
len(inputs), inputs[0].shape

(5, (2, 1027))

### 6.4.2 初始化模型参数
接下来初始化模型参数。隐藏单元个数`num_hiddens`是一个超参数。

In [15]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size 
# ctx = d2l.try_gpu() 
ctx = mx.cpu()
print('will use', ctx)

def get_params():
    def _one(shape):
        return nd.random.normal(scale=0.01, shape=shape, ctx=ctx)
    # 隐藏层参数
    W_xh = _one((num_inputs, num_hiddens)) 
    W_hh = _one((num_hiddens, num_hiddens)) 
    b_h = nd.zeros(num_hiddens, ctx=ctx) 
    # 输出层参数 
    W_hq = _one((num_hiddens, num_outputs)) 
    b_q = nd.zeros(num_outputs, ctx=ctx) 
    # 附上梯度 
    params = [W_xh, W_hh, b_h, W_hq, b_q] 
    for param in params:
        param.attach_grad() 
    return params

will use cpu(0)


### 6.4.3 定义模型
我们根据循环神经网络的计算表达式实现该模型。首先定义`init_rnn_state`函数来返回初始化的隐藏状态。它返回由一个形状为(批量大小, 隐藏单元个数)的值为0的NDArray组成的元组。使用元组是为了更便于处理隐藏状态含有多个NDArray的情况。

In [16]:
def init_rnn_state(batch_size, num_hiddens, ctx):
    return (nd.zeros(shape=(batch_size, num_hiddens), ctx=ctx), )

`rnn`函数定义了在一个时间步里如何计算隐藏状态和输出。这里的激活函数使用了`tanh`函数。`多层感知机`一节中介绍过，当元素在实数域上均匀分布时，`tanh`函数值的均值为0。

In [17]:
def rnn(inputs, state, params):
    # inputs和outputs皆为num_steps个形状为(batch_size, vocab_size)的矩阵 
    W_xh, W_hh, b_h, W_hq, b_q = params 
    H, = state 
    outputs = []
    for X in inputs:
        H = nd.tanh(nd.dot(X, W_xh) + nd.dot(H, W_hh) + b_h)
        Y = nd.dot(H, W_hq) + b_q
        outputs.append(Y) 
    return outputs, (H,)

做个简单的测试来观察输出结果的个数(时间步数)，以及第一个时间步的输出层输出的形状和隐藏状态的形状。

In [18]:
state = init_rnn_state(X.shape[0], num_hiddens, ctx)
inputs = to_onehot(X.as_in_context(ctx), vocab_size)
params = get_params()
outputs, state_new = rnn(inputs, state, params)
len(outputs), outputs[0].shape, state_new[0].shape

(5, (2, 1027), (2, 256))

### 6.4.4 定义预测函数
以下函数基于前缀`prefix`(含有数个字符的字符串)来预测接下来的`num_chars`个字符。这个函数稍显复杂，其中我们将循环神经单元`rnn`设置成了函数参数，这样在后面小节介绍其他循环神经网络时能重复使用这个函数。

In [19]:
# 本函数已保存在d2lzh包中⽅便以后使⽤
def predict_rnn(prefix, num_chars, rnn, params, init_rnn_state, num_hiddens, vocab_size, ctx, idx_to_char, char_to_idx):
    state = init_rnn_state(1, num_hiddens, ctx) 
    output = [char_to_idx[prefix[0]]] 
    for t in range(num_chars + len(prefix) - 1):
        # 将上⼀时间步的输出作为当前时间步的输⼊ 
        X = to_onehot(nd.array([output[-1]], ctx=ctx), vocab_size)
        # 计算输出和更新隐藏状态 
        (Y, state) = rnn(X, state, params) 
        # 下⼀个时间步的输⼊是prefix⾥的字符或者当前的最佳预测字符
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]]) 
        else:
            output.append(int(Y[0].argmax(axis=1).asscalar()))
    return ''.join([idx_to_char[i] for i in output])

我们先测试一下`predict_rnn`函数。我们将根据前缀`分开`创作长度为10个字符(不考虑前缀长度)的一段歌词。因为模型参数为随机值，所以预测结果也是随机的。

In [20]:
predict_rnn('分开', 10, rnn, params, init_rnn_state, num_hiddens, vocab_size, ctx, idx_to_char, char_to_idx)

'分开淡钩逗威琴吴抱嘛金枚'

### 6.4.5 裁剪梯度
RNN中较容易出现梯度衰减或梯度爆炸。我们会在`通过时间反向传播`一节中解释原因。为了应对梯度爆炸，我们可以`裁剪梯度`(clip gradient)。假设我们把所有模型参数梯度的元素拼接成一个向量 $\boldsymbol{g}$，并设裁剪的阈值是$\theta$。裁剪后的梯度
+ $\displaystyle \min\left(\frac{\theta}{|\boldsymbol{g}|}, 1\right)\boldsymbol{g}$

的$L_2$范数不超过$\theta$。

In [21]:
# 本函数已保存在d2lzh包中⽅便以后使⽤
def grad_clipping(params, theta, ctx):
    norm = nd.array([0], ctx) 
    for param in params:
        norm += (param.grad ** 2).sum() 
    norm = norm.sqrt().asscalar() 
    if norm > theta:
        for param in params: 
            param.grad[:] *= theta / norm

### 6.4.6 困惑度
我们通常使用`困惑度`(perplexity)来评价语言模型的好坏。`困惑度`是对`交叉熵损失函数`做指数运算后得到的值。特别地，
+ 最佳情况下，模型总是把标签类别的概率预测为1，此时困惑度为1
+ 最坏情况下，模型总是把标签类别的概率预测为0，此时困惑度为正无穷
+ 基线情况下，模型总是预测所有类别的概率都相同，此时困惑度为类别个数。

显然，**任何一个有效模型的困惑度必须小于类别个数。在本例中，困惑度必须小于词典大小`vocab_size`**。

### 6.4.7 定义模型训练函数
跟之前章节的模型训练函数相比，这里的模型训练函数有以下几点不同：
1. 使用困惑度评价模型
2. 在迭代模型参数前裁剪梯度
3. 对时序数据采用不同采样方法将导致隐藏状态初始化的不同。相关讨论可参考`语言模型数据集`一节

另外，考虑到后面将介绍的其他循环神经网络，为了更通用，这里的函数实现更长一些。

In [22]:
# 本函数已保存在d2lzh包中⽅便以后使⽤
def train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens, vocab_size, ctx, 
                          corpus_indices, idx_to_char, char_to_idx, is_random_iter, num_epochs, num_steps, 
                          lr, clipping_theta, batch_size, pred_period, pred_len, prefixes):
    # 随机采样还是相邻采样
    if is_random_iter:
        data_iter_fn = d2l.data_iter_random 
    else:
        data_iter_fn = d2l.data_iter_consecutive 
    params = get_params() 
    # 使用交叉熵损失函数
    loss = gloss.SoftmaxCrossEntropyLoss()
    # 迭代
    for epoch in range(num_epochs):
        if not is_random_iter: # 如使⽤相邻采样，在epoch开始时初始化隐藏状态
            state = init_rnn_state(batch_size, num_hiddens, ctx) 
        l_sum, n, start = 0.0, 0, time.time() 
        data_iter = data_iter_fn(corpus_indices, batch_size, num_steps, ctx) 
        for X, Y in data_iter:
            if is_random_iter: # 如使⽤随机采样，在每个⼩批量更新前初始化隐藏状态 
                state = init_rnn_state(batch_size, num_hiddens, ctx) 
            else: # 否则需要使⽤detach函数从计算图分离隐藏状态
                for s in state:
                    s.detach()
            with autograd.record():
                inputs = to_onehot(X, vocab_size) 
                # outputs有num_steps个形状为(batch_size, vocab_size)的矩阵 
                (outputs, state) = rnn(inputs, state, params) 
                # 拼接之后形状为(num_steps * batch_size, vocab_size) 
                outputs = nd.concat(*outputs, dim=0) 
                # Y的形状是(batch_size, num_steps)，转置后再变成⻓度为 
                # batch * num_steps 的向量，这样跟输出的⾏⼀⼀对应 
                y = Y.T.reshape((-1,)) 
                # 使⽤交叉熵损失计算平均分类误差 
                l = loss(outputs, y).mean()
            l.backward()
            grad_clipping(params, clipping_theta, ctx) # 裁剪梯度 
            d2l.sgd(params, lr, 1) # 因为误差已经取过均值，梯度不⽤再做平均 
            l_sum += l.asscalar() * y.size 
            n += y.size

        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % ( epoch + 1, math.exp(l_sum / n), time.time() - start)) 
            for prefix in prefixes:
                print(' -', predict_rnn(prefix, pred_len, rnn, params, init_rnn_state, num_hiddens, vocab_size, ctx, idx_to_char, char_to_idx))

### 6.4.8 训练模型并创作歌词
现在我们可以训练模型了。首先，设置模型超参数。我们将根据前缀`分开`和`不分开`分别创作长度为50个字符(不考虑前缀长度)的一段歌词。我们每过50个迭代周期便根据当前训练的模型创作一段歌词。

In [23]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2 
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']

下⾯采⽤随机采样训练模型并创作歌词。

In [24]:
train_and_predict_rnn(
    rnn, get_params, init_rnn_state, num_hiddens, vocab_size, ctx, 
    corpus_indices, idx_to_char, char_to_idx, True, num_epochs, num_steps, 
    lr, clipping_theta, batch_size, pred_period, pred_len, prefixes)

epoch 50, perplexity 69.924924, time 1.85 sec
 - 分开 我不要再 我有不外 我有了这 我有了这 我有了这 我有了这 我有了这 我有了这 我有了这 我有了这
 - 不分开 我想想这 我有我 别怪我 一子就 快什么 一颗四 我想要的可爱女人 坏坏的让我疯狂的可爱女人 坏坏
epoch 100, perplexity 10.045491, time 1.82 sec
 - 分开 一颗用双截棍的母斑鸠 印地的老斑  后悔你在不堡 一直我遇见你是一场悲剧 我可你的爱写在西元前 深
 - 不分开只 我不能再想 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 
epoch 150, perplexity 2.770006, time 1.83 sec
 - 分开 一只的步丽著我 家乡的爹笑早已苍老了 她地安 一点两著的画面的钟 从反方向开始移动 回到当初爱你的
 - 不分开扫 我不能再想 我不 我不 我不能 爱情走的太快就像龙卷风 不能承受我已无处可躲 我不要再想 我不能
epoch 200, perplexity 1.583634, time 1.82 sec
 - 分开 一只用双截的 哼哼年 一直走 我想就这样牵着你的手不放开 爱可不能够永远单纯没有悲害 你 靠着我的
 - 不分开吗 我叫你爸 你打我妈 这样中吗干嘛这样 还说让酒牵鼻子走 瞎 说一定实呵 有我 你和 征战 弓箭 
epoch 250, perplexity 1.293720, time 1.86 sec
 - 分开 我只无带 在小村外的溪边 默默等待 娘子 一壶好酒 再杰酸性抽在白  古么都武过堡 一枚忙红 再一
 - 不分开扫把的胖女巫 用拉丁文念咒语啦啦呜 她养的黑猫笑起来像哭 啦啦啦呜 是你的外婆 不场悲烛 还暖完日演


接下来采用相邻采样训练模型并创作歌词。

In [25]:
train_and_predict_rnn(
    rnn, get_params, init_rnn_state, num_hiddens, vocab_size, ctx, 
    corpus_indices, idx_to_char, char_to_idx, False, num_epochs, num_steps, lr,
    clipping_theta, batch_size, pred_period, pred_len, prefixes)

epoch 50, perplexity 59.043882, time 1.89 sec
 - 分开 我想要这 你谁了双 我想我这 你是我有 你想我 别怪我 别怪我的起 我不要你不 我不能再想 我不 
 - 不分开 我想要这 你谁了双 我想我这 你是我有 你想我 别怪我 别怪我的起 我不要你不 我不能再想 我不 
epoch 100, perplexity 6.743857, time 1.86 sec
 - 分开 我想能这样你 你你到 瞎直了你 想要一定 我想没定 我一定梦 不要再痛 你一定纵 不要不同 你一定
 - 不分开口 你已经离开我 不知不觉 我该了这节奏 后知后觉 我该了好节奏 后知后觉 我该了好节奏 后知后觉 
epoch 150, perplexity 2.000477, time 2.18 sec
 - 分开 我已啊 爱怎我到起画就像龙 用后一了蓝剧 我的完美主义 太彻底 分手的话像语言暴力 我已无能为力再
 - 不分开觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生
epoch 200, perplexity 1.306567, time 1.79 sec
 - 分开 我已的 爱 我种的事是 有我在抽汉 别窝在篮球 有窝什么奇 三窝三斗牛 有不懂不着 有话就什么 懂
 - 不分开觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生
epoch 250, perplexity 1.160314, time 1.93 sec
 - 分开 一候我 谁是么 我的耍任的微有 我只了我的微笑 我想要你的微笑 天来是的话休用 学说林跟武当怎 我
 - 不分开觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生
